In [ ]:
import os
import pandas as pd

# Ruta de la carpeta donde están todos tus CSVs
carpeta_csv = "ruta_a_tu_carpeta/Data"  # Cambia por tu ruta

# Crear una lista vacía para almacenar los DataFrames
dataframes = []

# Leer todos los archivos CSV en la carpeta y combinarlos
for archivo in os.listdir(carpeta_csv):
    if archivo.endswith(".csv"):
        df = pd.read_csv(os.path.join(carpeta_csv, archivo))
        dataframes.append(df)

# Combinar todos los DataFrames en uno solo
df_completo = pd.concat(dataframes, ignore_index=True)

# Visualizar las primeras filas
print(df_completo.head())

In [ ]:
# Características (amplitud de la señal)
X = df_completo[['amplitude']].values  # Convertir a un array de Numpy

# Etiquetas (ruido o sismo)
y = df_completo['label'].values  # 0 para ruido, 1 para sismo

# Normalizar los datos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)

# Dividir en segmentos (puedes ajustar el tamaño de la ventana según tus datos)
from sklearn.model_selection import train_test_split

# Usar una ventana de tiempo para dividir los datos en secuencias
def crear_secuencias(X, y, ventana):
    X_seq, y_seq = [], []
    for i in range(len(X) - ventana):
        X_seq.append(X[i:i + ventana])
        y_seq.append(y[i + ventana])
    return np.array(X_seq), np.array(y_seq)

# Definir el tamaño de la ventana (ejemplo: 100)
ventana_tiempo = 100
X_seq, y_seq = crear_secuencias(X_scaled, y, ventana_tiempo)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

print(f"Forma de los datos de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Forma de los datos de prueba: {X_test.shape}, {y_test.shape}")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Construir el modelo LSTM
model = Sequential()

# Primera capa LSTM
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Regularización para evitar overfitting

# Segunda capa LSTM
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Capa de salida
model.add(Dense(units=1, activation='sigmoid'))  # Usar 'sigmoid' para clasificación binaria

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Mostrar la arquitectura del modelo
model.summary()

In [ ]:
# Entrenar el modelo LSTM
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Pérdida en conjunto de prueba: {test_loss}')
print(f'Exactitud en conjunto de prueba: {test_accuracy}')

In [ ]:
model.save('modelo_lstm_sismos.h5')